### Chat Table Queries

SELECTs all chats and builds a dictionary of chat_id --> chat_info

In [7]:
chat_query = 'SELECT ROWID AS id, account_id, service_name AS service FROM chat'

### Handle Table

Selects all handles builds a dictionary mapping handle_ids to handle info (phone number)

In [8]:
handle_query = 'SELECT ROWID AS id, id AS full_number, country, service, uncanonicalized_id AS local_number, person_centric_id AS person_id FROM handle'

### Messages Table

Selects all messages and builds a list to be used for bucketing of messages to their corresponding chats / handles

In [9]:
msg_query = "SELECT ROWID AS id, text, is_from_me, datetime((date/1000000000) + strftime('%s', '2001-01-01 00:00:00'), 'unixepoch', 'localtime') AS date FROM message ORDER BY date DESC"

### Chat - Message Join Table

In [10]:
cmj_query = "SELECT cmj.message_id AS message_id, cmj.chat_id AS chat_id, text, is_from_me, handle_id, datetime((date/1000000000) + strftime('%s', '2001-01-01 00:00:00'), 'unixepoch', 'localtime') AS date FROM chat_message_join AS cmj INNER JOIN message AS m ON cmj.message_id = m.ROWID ORDER BY date"

### Chat - Handle Join Table

In [11]:
chj_query = "SELECT chj.handle_id AS handle_id, chj.chat_id AS chat_id, id AS full_number, country, service, uncanonicalized_id AS local_number, person_centric_id AS person_id FROM chat_handle_join AS chj INNER JOIN handle AS h ON chj.handle_id = h.ROWID ORDER BY handle_id"

### Chat - Handle - Message Query

#### NOTE
1. some messages have duplicates as iMessage/SMS in chat_message_join
2. some messages (presumably attachments) have NULL text values

In [12]:
combined_query = "SELECT m.ROWID AS message_id, cmj.chat_id AS chat_id, m.handle_id AS handle_id, h.id AS phone_number, text, is_from_me, style AS chat_style, m.service AS service, group_id, person_centric_id AS persion_id, datetime((date/1000000000) + strftime('%s', '2001-01-01 00:00:00'), 'unixepoch', 'localtime') AS date FROM message AS m INNER JOIN chat_message_join AS cmj ON cmj.message_id = m.ROWID INNER JOIN chat AS c ON cmj.chat_id = c.ROWID LEFT JOIN handle AS h ON m.handle_id = h.ROWID WHERE text IS NOT NULL GROUP BY message_id"

### New MessagesDB Class

In [1]:
from MessagesDB import MessagesDB

In [2]:
message_db = MessagesDB()

Connecting to Messages database file /Users/matthewwoop/Library/Messages/chat.db
Successfully connected -- booyah


In [3]:
chat_query = chat_query = 'SELECT ROWID AS id, account_id, service_name AS service FROM chat'
chat_result = message_db.query(chat_query)

In [4]:
chats = {c['id']: c for c in message_db.cursor.fetchall()}